<a href="https://colab.research.google.com/github/Jared-Steven/Collection-And-Analysis-Of-Tamil-Bible-Audio-And-Text-Data/blob/main/Collection_And_Analysis_Of_Tamil_Bible_Audio_And_Text_Data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Download The Necessary Libraries

In [ ]:
!pip install nltk
!pip install spacy
!pip install transformers
!pip install rippletagger
!pip install polyglot
!pip install indicnlp
!pip install inltk
!pip install indic-nlp-library
!pip install collections
!pip install stanfordnlp
!pip install --upgrade collections
!git clone https://github.com/anoopkunchukuttan/indic_nlp_resources.git
!git clone https://github.com/anoopkunchukuttan/indic_nlp_library.git

# Import The Necessary Libraries

In [ ]:
import requests
from bs4 import BeautifulSoup
import wave
import numpy as np
import matplotlib.pyplot as plt
from scipy.fft import fft, fftfreq
from scipy.signal import spectrogram, welch
import pandas as pd
import nltk
import spacy
from indicnlp import common
from indicnlp import loader
from indicnlp.tokenize import indic_tokenize
from indicnlp.tokenize import sentence_tokenize
import sys
from indicnlp import common
import json
import time
import os

# The below code is used to download the audio file of the Tamil Bible for all the books and its chapters from the website that you've provided through an open API End-Point on each of the Book's page.

In [ ]:
os.mkdir('/content/drive/MyDrive/bibaudio/Books') # Create A New folder to save the collected audio
url = "https://live.bible.is/bible/TAMWTC/MAT/1" # URL Of the Site to scrape the Name of the books and it's chapters
#rq_no = 1
request_result=requests.get(url)
time.sleep(10)
soup = BeautifulSoup(request_result.text, "html.parser")

s = soup.find('script', id='__NEXT_DATA__', type='application/json')
s_1 = s.text.strip()
data = json.loads(s_1)
data_1 = data['props']['pageProps']
books = data_1['books']

for i in range(len(books)):
  bookid = books[i]['book_id']
  chapters = books[i]['chapters']
  print('book_id: ', bookid)
  print('chapter: ', chapters)
  bknm_dir = f'/content/drive/MyDrive/bibaudio/Books/{bookid}' # Create A New folder for the seperate books of the new testament within the books folder to save the collected audio
  os.mkdir(bknm_dir)
  for j in range(len(chapters)):
    api_url = f"https://dbt.io/api/bibles/filesets/TCVWTCN2DA?key=a06351bd-4cae-40e7-90ec-ffdd13506987&v=4&book_id={bookid}&chapter_id={chapters[j]}&type=audio_drama"

    try:
        # Make a GET request to the API
        response = requests.get(api_url)

        time.sleep(5)

        # Check if the request was successful
        response.raise_for_status()  # Raises an HTTPError for bad responses (4xx and 5xx)

        # Parse the JSON response
        data = response.json()

        # Convert the JSON data to a string
        data_string = json.dumps(data)

        print(data_string)

        audlink = data['data'][0]['path']

        print('Link to the audio file', audlink)

        download_aud = requests.get(audlink)

        file_name = bknm_dir + f'/{bookid} - {chapters[j]}' + '.mp3'

        with open(file_name, 'wb') as file:
            file.write(download_aud.content)
            print(f'{file_name} is saved in the drive')


    except requests.exceptions.RequestException as e:
        print(f"An error occurred: {e}")

# The below Code is used to Scrape the text version of the Tamil bible

In [ ]:
os.mkdir('/content/drive/MyDrive/bibaudio/Books_Text')
url = "https://live.bible.is/bible/TAMWTC/MAT/1"

request_result=requests.get(url)
time.sleep(10)
soup = BeautifulSoup(request_result.text, "html.parser")

s = soup.find('script', id='__NEXT_DATA__', type='application/json')
s_1 = s.text.strip()
data = json.loads(s_1)
data_1 = data['props']['pageProps']
books = data_1['books']

for i in range(len(books)):
  bookid = books[i]['book_id']
  chapters = books[i]['chapters']
  print('book_id: ', bookid)
  print('chapter: ', chapters)
  bknm_dir = f'/content/drive/MyDrive/bibaudio/Books_Text/{bookid}'
  os.mkdir(bknm_dir)
  for j in range(len(chapters)):
    urls = f'https://live.bible.is/bible/TAMWTC/{bookid}/{chapters[j]}'
    max_attempts = 20
    for attempt in range(max_attempts):
        time.sleep(5)
        request_result=requests.get(urls)
        time.sleep(5)
        soup = BeautifulSoup(request_result.text, "html.parser")

        s = soup.find('script', id='__NEXT_DATA__', type='application/json')
        if s:
            s_1 = s.text.strip()
            data = json.loads(s_1)
            data_1 = data['props']['pageProps']
            current_chap = []

            try:
              chap = data_1['chapterText']
              current_chap.append(chap)
              bookid = current_chap[0][0]['book_id']
              book_nm = current_chap[0][0]['book_name']
              tamil_book_nm = current_chap[0][0]['book_name_alt']
              verse_no = current_chap[0][0]['verse_start_alt']
              verse_txt = current_chap[0][0]['verse_text']
              print(f'Bible Verses from {urls}: ')
              print('----- ------')
              print('bookid: ', bookid)
              print('book_nm: ', book_nm)
              print('tamil_book_nm: ', tamil_book_nm)
              print('--------------------------------------')
              print('verses')
              print('------')
              verses_no = []
              verses = []
              for k in range (len(current_chap[0])):
                  verse_no = current_chap[0][k]['verse_start_alt']
                  verse_txt = current_chap[0][k]['verse_text']
                  verses_no.append(verse_no)
                  verses.append(verse_txt)
                  output_file_path = bknm_dir + f"/{bookid} - {chapters[j]}" + ".txt"
              with open(output_file_path, "w") as file: #Write the data on the lists verses_no and verses into a text file
                  # Iterate through each element in the list
                  for v_n, vrs in zip(verses_no, verses):
                      # Remove any newline characters from the string
                      ent_verse = v_n + " " + vrs
                      cleaned_item = ent_verse.replace("\n", "")
                      # Write the cleaned string to the file with a newline character at the end
                      file.write(cleaned_item + "\n")

              print('verse_no: ', verses_no)
              print('Full Verses: ', verses)
              print('--------------------------------------')
              break
            except:
              print(f'Attempt {attempt} failed for the chapter url {urls}')

        else:
          time.sleep(5)
          break

# Data Preparation


In [ ]:
os.mkdir('/content/drive/MyDrive/bibaudio/books_in_WAV') # Create A New folder to save the collected audio

In [ ]:
def convert_mp3_to_wav(source_dir, target_dir):
    # Walk through the source directory
    for root, dirs, files in os.walk(source_dir):
        for file in files:
            if file.endswith('.mp3'):
                # Construct full file paths
                source_file_path = os.path.join(root, file)
                relative_path = os.path.relpath(root, source_dir)
                target_file_dir = os.path.join(target_dir, relative_path)
                target_file_path = os.path.join(target_file_dir, file[:-4] + '.wav')

                # Ensure the target directory exists
                os.makedirs(target_file_dir, exist_ok=True)

                # Load the MP3 file
                audio = AudioSegment.from_mp3(source_file_path)

                # Export as WAV file
                audio.export(target_file_path, format='wav')

                print(f'Converted: {source_file_path} to {target_file_path}')

# Example usage
source_directory = '/content/drive/MyDrive/bibaudio/Books'
target_directory = '/content/drive/MyDrive/bibaudio/books_in_WAV'

convert_mp3_to_wav(source_directory, target_directory)


# Mapping the text data with the audio data and saving it on a dictionary for quick access if needed.

In [ ]:
import os

def find_wav_and_text_files(directory):
    wav_files = []
    text_files = []

    for root, dirs, files in os.walk(directory):
        for file in files:
            if file.endswith('.wav'):
                wav_files.append(os.path.join(root, file))
            elif file.endswith('.txt'):
                text_files.append(os.path.join(root, file))

    return wav_files, text_files

def map_text_to_wav(directory1, directory2):
    wav_files1, text_files1 = find_wav_and_text_files(directory1)
    wav_files2, text_files2 = find_wav_and_text_files(directory2)

    wav_files = wav_files1 + wav_files2
    text_files = text_files1 + text_files2

    # Create a dictionary to map text files to wav files
    text_to_wav_mapping = {}

    for text_file in text_files:
        # Find the corresponding wav file
        base_name = os.path.splitext(os.path.basename(text_file))[0]
        corresponding_wav_file = next((wav_file for wav_file in wav_files if base_name in wav_file), None)

        if corresponding_wav_file:
            text_to_wav_mapping[text_file] = corresponding_wav_file

    return text_to_wav_mapping

# Example usage
directory1 = '/content/drive/MyDrive/bibaudio/Books_Text'
directory2 = '/content/drive/MyDrive/bibaudio/books_in_WAV'

mapping = map_text_to_wav(directory1, directory2)


# Print the mapping
for text_file, wav_file in mapping.items():
    print(f'Text File: {text_file} -> WAV File: {wav_file}')


# Exploratory Data Analysis

## Text Analysis

### Performing NLP tasks such as tokenization, part-of-speech tagging, named entity recognition, and sentiment analysis on the text data.

In [ ]:
# The path to the local git repo for Indic NLP library
INDIC_NLP_LIB_HOME=r"indic_nlp_library"

# The path to the local git repo for Indic NLP Resources
INDIC_NLP_RESOURCES=r"indic_nlp_resources"

# Add library to Python path
sys.path.append(r'{}\src'.format(INDIC_NLP_LIB_HOME))

# Set environment variable for resources folder
common.set_resources_path(INDIC_NLP_RESOURCES)

In [ ]:
# Ensure required resources are downloaded
nltk.download('punkt')
#spacy.cli.download("ta_core_news_sm")

# Function to read Tamil text file
def read_tamil_text(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        text = file.read()
    return text

file_path = '/content/drive/MyDrive/bibaudio/Books_Text/MAT/MAT - 1.txt'
tamil_text = read_tamil_text(file_path)

# Tokenization
indic_string = tamil_text
#print('Input String: {}'.format(indic_string))
print('Tokens: ')
for t in indic_tokenize.trivial_tokenize(indic_string):
        print(t)



In [ ]:
# Open the text file for reading
with open('/content/drive/MyDrive/bibaudio/Books_Text/MAT/MAT - 1.txt', 'r') as file:
    # Read all lines from the text file
    lines = file.readlines()

# Create a list to store the rows
rows = []

# Iterate over the lines
for line in lines:
    # Remove the newline character and split the line into values
    values = line.strip().split(',')
    rows.append(values)

# Open a new CSV file for writing
with open('output_file.csv', 'w', newline='') as file:
    import csv
    # Create a CSV writer object
    writer = csv.writer(file)

    # Write each row to the CSV file
    for row in rows:
        writer.writerow(row)

print("Text file converted to CSV successfully!")

In [ ]:
# Read the CSV file into a pandas DataFrame
df = pd.read_csv('/content/output_file.csv', header=0, sep='\t')

In [ ]:
# Assign a new column name to the first column
df.columns = [0] + [None] * (df.shape[1] - 1)  # Assign a name to the first column and keep others as None
df.rename(columns={0: 'verses'}, inplace=True)

# Print the column names
print(df.columns)

In [ ]:
# Print the first few rows of the DataFrame
print(df['verses'].head())

In [ ]:
df.info()
# Checking information of the data

In [ ]:
df.describe()
# Describe function shows us the frequency,unique and counts of all columns

In [ ]:
df.nunique()
# nunique() function return number of unique elements in the object.

# Audio Analysis

In [ ]:
# Load the WAV file
file_path = '/content/drive/MyDrive/bibaudio/books_in_WAV/MAT/MAT - 1.wav'
wav_file = wave.open(file_path, 'r')

# Extract basic information
n_channels = wav_file.getnchannels()
sample_width = wav_file.getsampwidth()
frame_rate = wav_file.getframerate()
n_frames = wav_file.getnframes()
duration = n_frames / frame_rate

print(f'Number of channels: {n_channels}')
print(f'Sample width (bytes): {sample_width}')
print(f'Frame rate (samples per second): {frame_rate}')
print(f'Number of frames: {n_frames}')
print(f'Duration (seconds): {duration:.2f}')

# Read frames and convert to numpy array
audio_frames = wav_file.readframes(n_frames)
audio_data = np.frombuffer(audio_frames, dtype=np.int16)
wav_file.close()

# If stereo, take only one channel for simplicity
if n_channels == 2:
    audio_data = audio_data[::2]

# Create time axis
time_axis = np.linspace(0, duration, num=len(audio_data))

# Plot the waveform
plt.figure(figsize=(12, 6))
plt.plot(time_axis, audio_data)
plt.title('Waveform')
plt.xlabel('Time (seconds)')
plt.ylabel('Amplitude')
plt.show()

# Perform FFT
fft_data = fft(audio_data)
fft_magnitude = np.abs(fft_data)
frequencies = fftfreq(len(audio_data), 1/frame_rate)

# Plot the FFT results
plt.figure(figsize=(12, 6))
plt.plot(frequencies[:len(frequencies)//2], fft_magnitude[:len(frequencies)//2])
plt.title('Frequency Analysis')
plt.xlabel('Frequency (Hz)')
plt.ylabel('Magnitude')
plt.show()

# Plot the spectrogram
frequencies_spec, times_spec, Sxx = spectrogram(audio_data, frame_rate)
plt.figure(figsize=(12, 6))
plt.pcolormesh(times_spec, frequencies_spec, 10 * np.log10(Sxx), shading='gouraud')
plt.title('Spectrogram')
plt.xlabel('Time (seconds)')
plt.ylabel('Frequency (Hz)')
plt.colorbar(label='Intensity (dB)')
plt.show()

# Plot Power Spectral Density (PSD)
frequencies_psd, psd = welch(audio_data, frame_rate, nperseg=1024)
plt.figure(figsize=(12, 6))
plt.semilogy(frequencies_psd, psd)
plt.title('Power Spectral Density')
plt.xlabel('Frequency (Hz)')
plt.ylabel('Power/Frequency (dB/Hz)')
plt.show()
